<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Nomusan_Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ノムさん度判定**
# <img src='https://ollama.com/public/ollama.png' alt="Ollama"/>
Huggingface のモデルだと時間がかかるので、別のLLM管理ソフト (Ollama) を使用します。

注１：関連 json ファイルを contens フォルダにアップロードしてください。

注２：動作がおかしくなったら 2. Ollama サーバのスタートからやり直してください。

In [1]:
# @title 1. コンポーネントのインストール
!curl https://ollama.ai/install.sh | sh
!pip install ollama

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os
# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  66982      0 --:--:-- --:--:-- --:--:-- 67075
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.

In [2]:
# @title 2. Ollama サーバの起動
import subprocess
proccess = subprocess.Popen(['ollama', 'serve'])

In [3]:
# @title 3. モデルの選択
model = "qwen2.5:7b" # @param {"type":"string"}
!ollama pull {model}

In [7]:
# @title 4. 動作確認
question = "あなたは誰ですか？" # @param {"type":"string"}
from IPython.display import display, Markdown
import ollama
response = ollama.chat(model, messages=[
  {
    'role': 'user',
    'content': question,
  },
])
#print(response['message']['content'])
display(Markdown(response['message']['content']))

私はQwenです、アリババクラウドによって作成された人工智能助手です。私は言語処理のタスクに取り組むことができ、文章生成、翻訳、質問回答などを行うことができます。あなたが何を知りたいかお聞かせください、できる限りのお手伝いをさせていただきます。

### 5. ノムさん度判定

In [8]:
JUDGE_SYSTEM = """あなたは、野村克也監督（ID野球）の思想・配球観・攻撃采配に精通した “審判AI（LLM-as-a-Judge）”です。

これから与える「状況」と「4つの回答（A/B/C/D）」を読み、以下の **5観点** で **0〜10点の整数値** で採点してください。

【評価観点】

1. nomura_like（野村監督らしさ）
  - ID野球の基礎（相性分析、確率思考、配球読み、データ重視、送りバントの慎重運用など）に沿うか
  - 「状況に応じた合理性」や「情報の非対称性の活用」が表現されているか
  - 野村監督の著書・インタビューに一致する思考様式か

2. tactical_quality（戦術としての期待値・合理性）
  - 回・アウトカウント・走者・打順・カウント・点差・相手投手の特徴などから見て得点期待値の高い采配になっているか
  - 作戦が矛盾していないか、過剰リスクや無意味なギャンブルがないか

3. format_quality（フォーマット遵守度）
  - 「作戦」「根拠（2文以内）」「具体的なサイン（1文）」の3行構成になっているか
  - 300〜400文字の制限内に収まっているか
  - 誤字・欠落・論理破綻がないか

4. explanation_clarity（説明の明快さ・理解性）
  - 結論と根拠の関係が明確で、論理的に筋が通っているか
  - 説明が抽象的すぎず、具体性と状況対応性があるか
  - 読み手が納得しやすい明快なロジックになっているか

5. instruction_following（指示遵守度）
  - プロンプトで指定された形式（行数・文字数・構成）を厳密に守っているか
  - 禁止表現（野球以外の話題、冗長な一般論、口語表現など）を避けているか
  - 出力全体から設計意図に従う姿勢が読み取れるか

【重要ルール】
- A/B/C/D のどれがどのモデルのものかは知らされていません。内容のみで判断してください。
- 採点はすべて 0〜10 の整数。
- 最も優れている回答を A/B/C/D のいずれか1つ選び、best_answer に記述。
- 必ずJSONのみを返し、説明・コメントは一切書かないこと。
- JSONは、必ず以下の形式で、**`"A"`, `"B"`, `"C"`, `"D"` をトップレベルのキーとして含めてください。**

【出力形式】
{
  "A":{"nomura_like":0,"tactical_quality":0,"format_quality":0,"explanation_clarity":0,"instruction_following":0},
  "B":{"nomura_like":0,"tactical_quality":0,"format_quality":0,"explanation_clarity":0,"instruction_following":0},
  "C":{"nomura_like":0,"tactical_quality":0,"format_quality":0,"explanation_clarity":0,"instruction_following":0},
  "D":{"nomura_like":0,"tactical_quality":0,"format_quality":0,"explanation_clarity":0,"instruction_following":0},
  "best_answer":"A"
}
"""

In [6]:
def call_judge_local(prompt, max_new_tokens=512):
    messages = [
        {"role": "system", "content": JUDGE_SYSTEM},
        {"role": "user", "content": prompt},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.0,
            do_sample=False
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # JSON開始位置から切り出し（安全策）
    json_start = decoded.rfind("{")
    return decoded[json_start:]

In [9]:
def call_judge_local(prompt, max_new_tokens=512):
    messages = [
        {"role": "system", "content": JUDGE_SYSTEM},
        {"role": "user", "content": prompt},
    ]
    outputs = ollama.chat(
        model = model,
        messages = messages,
        options = {"temperature": 0, "num_predict": max_new_tokens}
    )
    return outputs['message']['content']

In [10]:
import json

FILES = {
    "model_only": "/content/results_野村_model_only.json",
    "sft_only": "/content/results_野村_finetuning.json",
    "rule_rag": "/content/results_野村_lule.json",
    "sft_rule": "/content/results_野村_finetuning_RAGlule.json",
}

def load_results(path):
    with open(path, "r", encoding="utf-8") as f:
        return {x["id"]: x["output"] for x in json.load(f)}

loaded = {k: load_results(v) for k, v in FILES.items()}
ids = sorted(set.intersection(*[set(v.keys()) for v in loaded.values()]))

print("評価対象件数:", len(ids))

評価対象件数: 100


In [11]:
def build_judge_prompt(scenario, answers):
    return f"""
【試合状況】
{scenario}

【回答A】
{answers["A"]}

【回答B】
{answers["B"]}

【回答C】
{answers["C"]}

【回答D】
{answers["D"]}
"""

In [12]:
from tqdm import tqdm

results = []

for i in tqdm(ids):
    scenario = loaded["model_only"][i]  # 入力は共通想定

    answers = {
        "A": loaded["model_only"][i],
        "B": loaded["sft_only"][i],
        "C": loaded["rule_rag"][i],
        "D": loaded["sft_rule"][i],
    }

    prompt = build_judge_prompt(scenario, answers)
    judge_json = call_judge_local(prompt)

    # Add the model_map to the results list
    results.append({
        "id": i,
        "judge_raw": judge_json,
        "map": {
            "A": "model_only",
            "B": "sft_only",
            "C": "rule_rag",
            "D": "sft_rule",
        }
    })

100%|██████████| 100/100 [09:29<00:00,  5.69s/it]


In [13]:
with open("judge_results_" + model + ".json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("✅ Judge結果 保存完了")

✅ Judge結果 保存完了


In [14]:
import re
import json

def safe_json_load(text):
    """
    LLM出力から有効なJSONオブジェクトを抽出し、ロードを試みる。
    LLMが完全なJSONオブジェクトではないもの（例えば、一部のキーが欠落している、
    または余分なテキストが含まれている）を出力する可能性があるため、
    より堅牢な抽出ロジックを使用する。
    """
    if text is None:
        return None

    # 最も外側のJSONオブジェクトを探す
    match = re.search(r"(\{.*\})", text, re.DOTALL)
    if not match:
        return None

    potential_json_str = match.group(1)

    try:
        # まず、そのままロードを試みる
        return json.loads(potential_json_str)
    except json.JSONDecodeError as e:
        # ロードに失敗した場合、特定のLLMの出力形式に対応するためのヒューリスティックを試す
        # 観察された不正な形式: `{"key": value}, "best_answer":"X"`
        last_brace_comma_index = potential_json_str.rfind("},")
        if last_brace_comma_index != -1:
            first_part = potential_json_str[:last_brace_comma_index]
            remaining_part = potential_json_str[last_brace_comma_index+2:].strip()

            # Now remaining_part might be something like '"best_answer":"A"}'
            # We need to extract the key-value pair and ensure it's valid.
            if remaining_part.endswith('}'):
                extracted_second_part = remaining_part[:-1].strip() # Remove the final '}'
            else:
                extracted_second_part = remaining_part.strip()

            if extracted_second_part.startswith('"best_answer":'):
                repaired_json_str = f"{first_part}, {extracted_second_part}}}"
                try:
                    return json.loads(repaired_json_str)
                except json.JSONDecodeError:
                    pass

        # その他のJSONDecodeErrorの場合、または修復が失敗した場合
        return None

In [15]:
import json
import pandas as pd

rows = []
score_cols = [
    "nomura_like",
    "tactical_quality",
    "format_quality",
    "explanation_clarity",
    "instruction_following"
]

for r in results:
    j = safe_json_load(r["judge_raw"])
    if j is None:
        print(f"Skipping entry {r['id']} due to JSON parsing error.")
        continue

    model_map = r["map"]

    # Check if 'best_answer' exists in the parsed JSON
    if "best_answer" not in j:
        print(f"Skipping entry {r['id']} as 'best_answer' key is missing in LLM output: {j}")
        continue

    best_k = j["best_answer"]

    for k in ["A", "B", "C", "D"]:
        row_data = {
            "id": r["id"],
            "model_key": model_map[k],
        }
        if k == best_k:
            # For the best answer, extract its specific scores from j[best_k]
            # Ensure j[best_k] exists and is a dictionary
            if isinstance(j.get(best_k), dict):
                for col in score_cols:
                    row_data[col] = j[best_k].get(col, 0)
            else:
                # If the best_k entry itself is malformed, assign 0s
                for col in score_cols:
                    row_data[col] = 0
        else:
            # For other answers, assign 0 for all scores
            for col in score_cols:
                row_data[col] = 0
        rows.append(row_data)

# Create DataFrame from rows (now populated correctly)
df = pd.DataFrame(rows)

# Ensure score columns are numeric
for c in score_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Calculate mean only for the identified score columns
mean_scores = df[score_cols].mean().round(3)
mean_scores.to_csv(
    "judge_mean_scores.csv",
    encoding="utf-8-sig"
)

print("✅ 平均スコアCSV 出力完了")
display(mean_scores)

✅ 平均スコアCSV 出力完了


,0
nomura_like,2.372
tactical_quality,2.128
format_quality,2.485
explanation_clarity,2.388
instruction_following,2.500


In [20]:
df.to_csv("judge_results"+model+".csv", index = False)